In [2]:
import sys, os
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QLineEdit, QInputDialog, QLabel, QVBoxLayout
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import Qt ,QCoreApplication
import numpy as np
from time import sleep
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)


def maple_API(key):
    # url, API key 입력
    url = 'https://public.api.nexon.com/openapi/maplestory/v1/cube-use-results'

    # 시작일,종료일 설정
    start = "2023-2-20" # 최초 데이터 시작일
    last = (datetime.today() - timedelta(days=1)).strftime(format="%Y-%m-%d")

    start_date = datetime.strptime(start, "%Y-%m-%d")
    last_date = datetime.strptime(last, "%Y-%m-%d")

    # 종료일 까지 반복
    total_cnt = 0
    total_history = []
    while start_date <= last_date:
        dates = start_date.strftime("%Y-%m-%d")
        headers = {
            'authorization' : key
        }
        params = {
            'count' : 1000,
            'date' : dates,
            'cursor' : ''
        }
        res = requests.get(url,headers=headers, params=params)
        data = res.json()
        if "count" in data:
            total_cnt += data['count']

        if len(data['cube_histories']) > 0:
            total_history.append(data['cube_histories'])

        start_date += timedelta(days=1)

    final_output = []
    for history in total_history:
        character_name = []
        create_date = []
        cube_type = []
        miracle_time_flag = []
        item_equip_part = []
        item_level = []
        target_item = []
        potential_option_grade = []
        additional_potential_option_grade = []
        before_potential_options = []
        before_additional_potential_options = []
        after_potential_options = []
        after_additional_potential_options = []
        #filtered_legend = []
        for i in history:
            character_name.append(i['character_name'])
            create_date.append(i['create_date'])
            cube_type.append(i['cube_type'])
            miracle_time_flag.append(i['miracle_time_flag'])
            item_equip_part.append(i['item_equip_part'])
            item_level.append(i['item_level'])
            target_item.append(i['target_item'])
            potential_option_grade.append(i['potential_option_grade'])
            additional_potential_option_grade.append(i['additional_potential_option_grade'])
            before_potential_options.append(i['before_potential_options'])
            before_additional_potential_options.append(i['before_additional_potential_options'])
            after_potential_options.append(i['after_potential_options'])
            after_additional_potential_options.append(i['after_additional_potential_options'])
            #for value, grade in i['after_potential_options'].items():
                #if grade == '레전드리':
                    #filtered_legend.append(1)

        output = pd.DataFrame({
            'character_name'  : character_name,
            'create_date' : create_date,
            'cube_type' : cube_type,
            'miracle_time_flag' : miracle_time_flag,
            'item_equip_part' : item_equip_part,
            'item_level' : item_level,
            'target_item' : target_item,
            'potential_option_grade' : potential_option_grade,
            'additional_potential_option_grade' : additional_potential_option_grade,
            'before_potential_options' : before_potential_options,
            'before_additional_potential_options' : before_additional_potential_options,
            'after_potential_options' : after_potential_options,
            'after_additional_potential_options' : after_additional_potential_options,
            #'filtered_legend' : filtered_legend
        })
        final_output.append(output)
    final_output = pd.concat(final_output)
    
    s_cu = (final_output['cube_type']=='수상한 큐브').sum()
    j_cu = (final_output['cube_type']=='장인의 큐브').sum()
    m_cu = (final_output['cube_type']=='명장의 큐브').sum()
    r_cu = (final_output['cube_type']=='레드 큐브').sum()
    b_cu = (final_output['cube_type']=='블랙 큐브').sum()
    se_cu = (final_output['cube_type']=='수상한 에디셔널 큐브').sum()
    e_cu = (final_output['cube_type']=='에디셔널 큐브').sum()
    cube_info = [s_cu,j_cu,m_cu,r_cu,b_cu,se_cu,e_cu]
    #filtered_legend = (final_output['filtered_legend']).sum()
    #legend = (final_output['after_potential_options'])
    #final_output.to_csv('Maple_API_Cube_utf-8.csv', index=False)
    final_output.to_csv('Maple_API_Cube_euc-kr.csv', index=False,encoding='euc-kr')
    return total_cnt, cube_info, #filtered_legend

    
print("메이플 큐브 사용량 API")
print("API key를 입력해주세요.")
key = input()
total_cnt, cube_info = maple_API(key)
print("총 큐브 사용량 : ", total_cnt)
print("수상한 큐브 : ", cube_info[0])
print("장인의 큐브 : ", cube_info[1])
print("명장의 큐브 : ", cube_info[2])
print("레드 큐브 : ", cube_info[3])
print("블랙 큐브 : ", cube_info[4])
print("수상한 에디셔널 큐브 : ", cube_info[5])
print("에디셔널 큐브 : ", cube_info[6])
#print("블랙 큐브 레전 등업횟수 : ", filtered_legend)



메이플 큐브 사용량 API
API key를 입력해주세요.
총 큐브 사용량 :  2753
수상한 큐브 :  1412
장인의 큐브 :  417
명장의 큐브 :  7
레드 큐브 :  128
블랙 큐브 :  572
수상한 에디셔널 큐브 :  169
에디셔널 큐브 :  0
